In [1]:
import datetime as dt
import holidays
import requests as req
import pandas as pd
import yfinance as yf
import sqlite3 as sqlite

In [2]:
def GET_INDICE_COMPONENTS():
    res  = req.get('https://www.slickcharts.com/sp500', headers={ 'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36', 'X-Requested-With':'XMLHttpRequest' })
    page = pd.read_html(res.content)
    tbl  = page[0]
    return tbl

SNP_500 = GET_INDICE_COMPONENTS()['Symbol']

In [3]:
ETFS        = ['SPY','DIA','QQQ','XLF','XLC','XLE','XLU','XLP','XLK','XLV','XLI','XLY','XLB','XOP','XBI','XRT','XHB','XME','XLRE','IWM','IYR','NOBL','AMLP','OIH','KRE','VNQ','ITB','KBE','IBB','SMH']
TICKERS     = [*ETFS, *SNP_500.head(50)]
TODAY       = dt.date.today()
IS_HOLIDAY  = TODAY in holidays.US(years=TODAY.year)

In [4]:
if IS_HOLIDAY: raise Exception('IS_HOLIDAY')

In [5]:
def GET_AF_OPT_CHAINS(TICKERS):

    # ================ Helpers ================ #
    def _scrap(TICKER, EXPIRY, EXCHANGES=['NYSE','NASDAQ','AMEX'], STRIKES=50):
        for i, EXCHANGE in enumerate(EXCHANGES):
            try: 
                return pd.read_html(req.get(f"https://www.advfn.com/stock-market/{EXCHANGE}/{TICKER}/options?expiry={EXPIRY}&strikes={STRIKES}", headers={ 'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36', 'X-Requested-With':'XMLHttpRequest' }).content)
            except: 
                if i+1 == len(EXCHANGES): 
                    return [pd.DataFrame(), pd.DataFrame()]
    

    # ================ Main ================ #
    Pipe = []
    for TICKER in TICKERS:
        Asset = yf.Ticker(TICKER)
        price = Asset.info.get('previousClose')

        for EXP in Asset.options:
            page = _scrap(TICKER, EXP.replace('-','')) 
            for OPT in [0,1]: 
                try: 
                    Opt = page[OPT]
                    Pipe.append(pd.DataFrame({
                        'Date':     TODAY, 
                        'Ticker':   TICKER, 
                        'Price':    price, 
                        'Opt':      { 0:'C', 1:'P' }[OPT], 
                        'Expiry':   EXP, 
                        'Strike':   Opt['Strike'], 
                        'Vol':      Opt['Volume'], 
                        'OI':       Opt['OPEN INT'], 
                        'Ask':      Opt['Ask Price'], 
                        'Mid':      Opt['Midpoint'], 
                        'Bid':      Opt['Bid Price'], 
                        'Last':     Opt['Last Price'], 
                    }))
                except Exception as E: print(E)
            pass
        pass
    pass
    return pd.concat(Pipe, ignore_index=1)

Chains = GET_AF_OPT_CHAINS(TICKERS)
# Chains

/tmp/ipykernel_1766/2709542142.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(Pipe, ignore_index=1)


In [6]:
con = sqlite.connect('Filebase.db')
Chains.to_sql('Chain', con, if_exists='append', index=0)

135814